In [1]:
import pandas as pd

df = pd.read_json("hf://datasets/AGBonnet/augmented-clinical-notes/augmented_notes_30K.jsonl", lines=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
# Pip installations
!pip install transformers
!pip install torch
!pip install scikit-learn
!pip install pandas
!pip install pytorch-lightning
!pip install datasets rouge-score
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
# Import necessary libraries and packages
import numpy as np
import torch
import evaluate
from sklearn.model_selection import train_test_split
import pytorch_lightning as pl
from torch.utils.data import Dataset,DataLoader
from nltk.translate.bleu_score import sentence_bleu
from transformers import BartTokenizer, BartForConditionalGeneration, get_linear_schedule_with_warmup
from torch.optim import AdamW

In [4]:
df.head()

,note,conversation,idx,summary,full_note
0,"A a sixteen year-old girl, presented to our Ou...","Doctor: Good morning, what brings you to the O...",155216,"{\n""visit motivation"": ""Discomfort in the neck...","A a sixteen year-old girl, presented to our Ou..."
1,This is the case of a 56-year-old man that was...,"Doctor: Hi, how are you feeling today?\nPatien...",77465,"{\n""visit motivation"": ""Complaints of a dull p...",This is the case of a 56-year-old man that was...
2,A 36-year old female patient visited our hospi...,"Doctor: Hello, what brings you to the hospital...",133948,"{\n""visit motivation"": ""Pain and restricted ra...",A 36-year old female patient visited our hospi...
3,A 49-year-old male presented with a complaint ...,"Doctor: Good morning, Mr. [Patient's Name]. I'...",80176,"{\n""visit motivation"": ""Pain in the left proxi...",A 49-year-old male presented with a complaint ...
4,A 47-year-old male patient was referred to the...,"Doctor: Good morning, how are you feeling toda...",72232,"{\n""visit motivation"": ""Recurrent attacks of p...",A 47-year-old male patient was referred to the...


In [5]:
# prompt: make a variable df_full that is a sample of df

# df_full = df.sample(frac=0.1, random_state=42) # Sample 10% of the dataframe


In [6]:

# df_full = df
# Load the tokenizer
tokenizer= BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Ensure all content is string type and handle missing values
df_full['full_note']= df_full['full_note'].fillna('').astype(str)
df_full['summary']= df_full['summary'].fillna('').astype(str)

# Now tokenize the article content and summary
df_full['full_note']= df_full['full_note'].apply(lambda x: tokenizer.batch_encode_plus(
    [x], max_length= 512, padding= 'max_length', truncation= True, return_tensors= 'pt'))

df_full['summary']= df_full['summary'].apply(lambda x: tokenizer.batch_encode_plus(
    [x], max_length= 150, padding= 'max_length', truncation= True, return_tensors= 'pt'))

# View tokenized data
print(df_full[['full_note', 'summary']].head())

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

NameError: name 'df_full' is not defined

In [ ]:
# Perform train-test split (80% training, 20% validation)
train_df, val_df= train_test_split(df_full, test_size=0.2, random_state=42)

# Print the size of each set
print(f'Training set size: {len(train_df)}')
print(f'Validation set size: {len(val_df)}')

In [ ]:
# Define a class that helps load batches of tokenized data
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, source_len, target_len):
        self.tokenizer= tokenizer
        self.data= data
        self.source_len= source_len
        self.target_len= target_len
        self.source_text= data['full_note']
        self.target_text= data['summary']

    def __len__(self):

        # Return the number of examples
        return len(self.target_text)

    def __getitem__(self, index):

        # Get the source (article content) and target (summary) text
        source_text= str(self.source_text[index])
        target_text= str(self.target_text[index])

        # Tokenize the source and target text
        source= self.tokenizer.batch_encode_plus(
            [source_text],
            max_length= self.source_len,
            padding= 'max_length',
            truncation= True,
            return_tensors= 'pt'
        )

        target= self.tokenizer.batch_encode_plus(
            [target_text],
            max_length= self.target_len,
            padding= 'max_length',
            truncation= True,
            return_tensors= 'pt'
        )

        # Return the tokenized input and target
        return {
            'source_ids': source['input_ids'].squeeze(),
            'source_mask': source['attention_mask'].squeeze(),
            'target_ids': target['input_ids'].squeeze(),
            'target_mask': target['attention_mask'].squeeze()
        }

In [ ]:
# Define a function to create data loaders for the training and validation sets
def create_data_loader(dataframe, tokenizer, source_len, target_len, batch_size):
    dataset= CustomDataset(
        data= dataframe,
        tokenizer= tokenizer,
        source_len= source_len,
        target_len= target_len
    )

    return DataLoader(dataset, batch_size= batch_size, shuffle= True, num_workers= 4)

# Define parameters for the DataLoader
BATCH_SIZE= 8
SOURCE_LEN= 512
TARGET_LEN= 150

# Reset the indices of both training and validation sets
train_df= train_df.reset_index(drop= True)
val_df= val_df.reset_index(drop= True)

# Create training and validation data loaders
train_data_loader= create_data_loader(train_df, tokenizer, SOURCE_LEN, TARGET_LEN, BATCH_SIZE)
val_data_loader= create_data_loader(val_df, tokenizer, SOURCE_LEN, TARGET_LEN, BATCH_SIZE)

# Check the DataLoader output
data= next(iter(train_data_loader))
print(data.keys())
print(f"Source_ids shape: {data['source_ids'].shape}")
print(f"Target_ids shape: {data['target_ids'].shape}")

In [ ]:
# Define the Abstractive Summarization Model using PyTorch Lightning
class AbstractiveSummarizationModel(pl.LightningModule):

    # Initialize the model and tokenizer
    def __init__(self, model, tokenizer):
        super(AbstractiveSummarizationModel, self).__init__()
        self.model= model  # BART model for summarization
        self.tokenizer= tokenizer  # Tokenizer used to preprocess text data

    # Forward pass - how the model processes input and generates output
    def forward(self, input_ids, attention_mask, decoder_input_ids, decoder_attention_mask=None):
        outputs= self.model(
            input_ids= input_ids,  # Tokenized article content
            attention_mask= attention_mask,  # Mask to focus on real tokens (ignore padding)
            decoder_input_ids= decoder_input_ids,  # Tokenized summary (target text)
            decoder_attention_mask= decoder_attention_mask  # Mask to ignore padding in summary
        )
        return outputs.loss, outputs.logits  # Return loss and model output (logits)

    # Define one step of training - how the model learns from the data
    def training_step(self, batch, batch_idx):
        input_ids= batch['source_ids']  # Tokenized articles (input)
        attention_mask= batch['source_mask']  # Attention mask for the input
        decoder_input_ids= batch['target_ids']  # Tokenized summaries (target)
        decoder_attention_mask= batch['target_mask']  # Attention mask for the summaries

        # Perform forward pass for training
        loss, outputs= self(
            input_ids= input_ids,
            attention_mask= attention_mask,
            decoder_input_ids= decoder_input_ids,
            decoder_attention_mask= decoder_attention_mask
        )

        # Ensure that the loss is not None before logging
        if loss is not None:
            self.log("train_loss", loss)
        return loss

    # Define one step of validation - how the model evaluates its performance
    def validation_step(self, batch, batch_idx):
        input_ids= batch['source_ids']  # Tokenized articles (input)
        attention_mask= batch['source_mask']  # Attention mask for the input
        decoder_input_ids= batch['target_ids']  # Tokenized summaries (target)
        decoder_attention_mask= batch['target_mask']  # Attention mask for the summaries

        # Perform forward pass for validation
        loss, outputs= self(
            input_ids=input_ids,
            attention_mask= attention_mask,
            decoder_input_ids= decoder_input_ids,
            decoder_attention_mask= decoder_attention_mask
        )

        # Ensure that loss is not None before logging
        if loss is not None:
            self.log('val_loss', loss)
        return loss

    # Define the optimizer - AdamW optimizer with learning rate of 3e-5
    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr= 3e-5)

In [ ]:
# Load the pretrained BART model for conditional generation
bart_model= BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

# Instantiate the summarization model we defined earlier
model= AbstractiveSummarizationModel(model= bart_model, tokenizer= tokenizer)

# Determine whether a GPU is available
accelerator= 'gpu' if torch.cuda.is_available() else 'cpu'
devices= 1

# Initialize the PyTorch Lightning trainer
trainer= pl.Trainer(
    max_epochs= 3,
    devices= devices,
    accelerator= accelerator,
    enable_progress_bar= True
)

# Start the training process
trainer.fit(model, train_data_loader, val_data_loader)

In [ ]:
# Define the summarization function based on ProjectPro
def summarize_article(article):

    # Load BART model and tokenizer
    model_name= 'facebook/bart-large-cnn'
    tokenizer= BartTokenizer.from_pretrained(model_name)
    model= BartForConditionalGeneration.from_pretrained(model_name)

    # Tokenize and encode the article
    inputs= tokenizer.encode(article, return_tensors= 'pt', max_length= 1024, truncation= True)

    # Generate summary
    summary_ids= model.generate(inputs, num_beams= 4, max_length= 250, early_stopping= True)

    # Decode the summary with explicit clean_up_tokenization_spaces set
    summary= tokenizer.decode(summary_ids[0], skip_special_tokens= True, clean_up_tokenization_spaces= True)

    return summary

# Example usage
article= '''A a sixteen year-old girl, presented to our Outpatient department with the complaints of discomfort in the neck and lower back as well as restriction of body movements. She was not able to maintain an erect posture and would tend to fall on either side while standing up from a sitting position. She would keep her head turned to the right and upwards due to the sustained contraction of the neck muscles. There was a sideways bending of the back in the lumbar region. To counter the abnormal positioning of the back and neck, she would keep her limbs in a specific position to allow her body weight to be supported. Due to the restrictions with the body movements at the neck and in the lumbar region, she would require assistance in standing and walking. She would require her parents to help her with daily chores, including all activities of self-care.\nShe had been experiencing these difficulties for the past four months since when she was introduced to olanzapine tablets for the control of her exacerbated mental illness. This was not her first experience with this drug over the past seven years since she had been diagnosed with bipolar affective disorder. Her first episode of the affective disorder was that of mania at the age of eleven which was managed with the use of olanzapine tablets in 2.5–10 mg doses per day at different times. The patient developed pain and discomfort in her neck within the second week of being put on tablet olanzapine at a dose of 5 mg per day. This was associated with a sustained and abnormal contraction of the neck muscles that would pull her head to the right in an upward direction. These features had persisted for the first three years of her illness with a varying intensity, distress, and dysfunction which would tend to correlate with the dose of olanzapine. Apart from a brief period of around three weeks when she was given tablet trihexyphenidyl 4 mg per day for rigidity in her upper limbs, she was not prescribed any other psychotropic medication. The rigidity showed good response to this medication which was subsequently stopped. The introduction and subsequent withdrawal of this medication did not bring about any change in the sustained abnormal contraction of her neck muscles.\nImprovement and subsequent remission of the mood symptoms of the patient provided the treatment team with an opportunity to stop olanzapine. The discomfort in the neck and the abnormal movement of the neck muscles persisted over the next three months’ period when she was off olanzapine without any significant change, even with a trial of propranolol, trihexyphenidyl, and phenargan injection. Reintroduction of olanzapine (at a dose of 2.5 mg per day) after a gap of three months for the reemergence of some behavioral features led to a slight aggravation of the already existing abnormal movement and posturing of the neck.\nWith improvement in the clinical picture, olanzapine was reduced and stopped. She was put on tablet sodium valproate, 1000 mg per day during this period for the stabilization of her mood when she was also given escitalopram for a period of three months for her depressive features. The patient responded well to this change in medication, but she developed amenorrhea for which no cause was established after a detailed gynecological evaluation. Keeping in mind the possibility of valproate-induced menstrual disturbance, she was shifted to tablet lithium 450 mg per day. The patient was well maintained on this medication for a period of around two years. However, she developed hypothyroidism for which eltroxin was introduced at a dose of 50 micrograms per day.\nDuring this period of two years and seven months, the abnormal contraction of the neck muscles and the abnormal positioning of the head improved slightly, and with the improvement, it would cause less discomfort and interference in her activities. However, these movements failed to disappear completely. Another exacerbation of the mood symptoms in the form of mania warranted a need for the introduction of olanzapine (by a different treatment team) and the patient was reintroduced to 10 mg olanzapine on a daily basis, which led to the current presentation as described earlier.\nAfter the case was seen at our institute, the psychotropic medications were stopped as her mood symptoms had remitted and she was put on tablet tetrabenazine (built up to 75 mg per day in divided doses) with which the patient had started showing some response with an improvement in abnormal movements of the muscles of the neck as well as the back. She is now able to stand with support and can do some daily chores on her own. The pain and discomfort in the back and neck have also reduced.\nDuring the course of the illness, the patient has been investigated for the presence of any neurological illness as the cause of her abnormal movements. Her MRI scan of the brain, serum and urine copper levels, slit lamp microscopy for the KF ring, complete blood count, TLC, DLC, and USG of the abdomen did not reveal any abnormalities. Her thyroid function tests were deranged subsequent to the introduction of tablet lithium carbonate which was restored to normal after the introduction of tablet eltroxin.\nDystonia is a syndrome of sustained muscle contractions that produce twisting and repetitive movements or abnormal postures. The descriptions of the extent and severity of muscle involvement are variable, ranging from intermittent contraction limited to a single body region, to generalized dystonia involving the limbs and axial muscles.\nEver since the introduction of the term, “dystonia” by Oppenhiem in the early part of the twentieth century, it has been an area of focused attention of the neurologists. In 1973, Keegan and Rajput introduced the term, “” to describe drug-induced, sustained muscle spasm causing repetitive movements or abnormal postures. “” was a term introduced by Burke in 1982, the description of which required the presence of chronic dystonia, a history of antipsychotic drug treatment preceding or concurrent with the onset of dystonia, the exclusion of known causes of secondary dystonia by appropriate clinical and laboratory evaluation, and a negative family history of dystonia for definitive diagnosis.\nThe dystonia could be classified based on the region(s) of the body involved. Involvement of isolated regions like the face, neck, and arms would be labeled as focal dystonia, whereas simultaneous involvement of two or more contiguous areas would be called segmental dystonia. When the clinical picture is that of involvement of two or more noncontiguous regions, the label used is, “multifocal” and the involvement of one leg and one other body region makes it the generalized type.\nThe symptoms of tardive dystonia could begin even after a few days or weeks of exposure to the offending agent. Tardive dystonia is prevalent in 0.5–21.6% of the patients who are treated with neuroleptics.\nThe syndrome of tardive dystonia has been reported with most of the typical antipsychotics.[] It has been associated with the atypical antipsychotics, namely risperidone, olanzapine, quetiapine, and aripiprazole. Reports of tardive dystonia developing with the use of atypical antipsychotics have been predominantly in the cases of nonaffective psychosis and in the adult population in the age ranges of the midthirties and forties. Our case is the first case of an affective illness in an adolescent girl developing tardive dystonia on olanzapine. The aggravation of the clinical features with the inadvertent reintroduction of the medication suggests olanzapine is the offending agent. With the growing acceptance of olanzapine as the first-line therapy for the manic phase of bipolar illness and as a mood stabilizer for the maintenance therapy, one needs to be cautious about the emergence of this troublesome adverse effect of this therapy. The patient has shown some response to the introduction of tetrabenazine.'''
summary= summarize_article(article)
print('Summary:')
print(summary)

In [ ]:
# Define reference summary for comparison (human-written summary)
reference_summary= '''Visit Motivation:
The patient visited due to discomfort in the neck and lower back, restriction of body movements, inability to maintain an erect posture, and requiring assistance in standing and walking.

Admission:
There were no prior admissions recorded.

Patient Information:
The patient is a sixteen-year-old female. Ethnicity, weight, height, family medical history, recent travels, socio-economic context, and occupation are not available.

Patient Medical History:
There are no notable physiological contexts reported. Psychologically, the patient was diagnosed with bipolar affective disorder at the age of eleven, with the first episode being mania. Vaccination history, allergies, exercise frequency, nutrition, sexual history, alcohol consumption, drug usage, and smoking status are not documented.

Surgeries:
There is no history of surgeries.

Symptoms:
The patient reports discomfort in the neck and lower back, restriction of body movements, and inability to maintain an erect posture. Symptoms have been present for the past four months. The head is turned to the right and upwards due to sustained contraction of neck muscles, and there is sideways bending of the back in the lumbar region. Limbs are positioned to support body weight, and standing up from a sitting position affects the symptoms.

Medical Examinations:
No medical examinations are documented.

Diagnosis Tests:
No diagnostic tests are reported.

Treatments:

    The patient has been taking Olanzapine tablets (5 mg per day) daily for the past four months to control exacerbated mental illness. However, treatment has led to pain and discomfort in the neck, sustained and abnormal contraction of neck muscles, and the need for assistance in daily chores. The patient has a history of taking olanzapine doses between 2.5–10 mg per day over the past seven years.

    Additionally, the patient was prescribed Trihexyphenidyl (4 mg per day) for a brief period of around three weeks to address rigidity in the upper limbs. The response to this treatment was good.

Discharge:
No discharge reason, referral, follow-up, or discharge summary has been provided..'''

# Generate model summary
generated_summary= summarize_article(article)

print('Generated Summary:')
print(generated_summary)

# Load the ROUGE metric using the evaluate library
rouge= evaluate.load('rouge')

# Calculate the ROUGE score
scores= rouge.compute(predictions=[generated_summary], references=[reference_summary])

# Print ROUGE scores
print('\nROUGE scores:')
print(scores)


In [ ]:
# Improve model summary generation
def summarize_article_with_tweaks(article):

    # Load BART model and tokenizer
    model_name= 'facebook/bart-large-cnn'
    tokenizer= BartTokenizer.from_pretrained(model_name)
    model= BartForConditionalGeneration.from_pretrained(model_name)

    # Tokenize and encode the article
    inputs= tokenizer.encode(article, return_tensors= 'pt', max_length= 1024, truncation= True)

    # Generate summary with tweaked parameters
    summary_ids= model.generate(
        inputs,
        num_beams= 6,               # Increased beams for better quality
        max_length= 150,
        repetition_penalty= 1.2,    # Add repetition penalty to avoid repeated phrases
        early_stopping= True
    )

    # Decode the summary
    summary= tokenizer.decode(summary_ids[0], skip_special_tokens= True, clean_up_tokenization_spaces= True)

    return summary

# Example usage
article= '''A a sixteen year-old girl, presented to our Outpatient department with the complaints of discomfort in the neck and lower back as well as restriction of body movements. She was not able to maintain an erect posture and would tend to fall on either side while standing up from a sitting position. She would keep her head turned to the right and upwards due to the sustained contraction of the neck muscles. There was a sideways bending of the back in the lumbar region. To counter the abnormal positioning of the back and neck, she would keep her limbs in a specific position to allow her body weight to be supported. Due to the restrictions with the body movements at the neck and in the lumbar region, she would require assistance in standing and walking. She would require her parents to help her with daily chores, including all activities of self-care.\nShe had been experiencing these difficulties for the past four months since when she was introduced to olanzapine tablets for the control of her exacerbated mental illness. This was not her first experience with this drug over the past seven years since she had been diagnosed with bipolar affective disorder. Her first episode of the affective disorder was that of mania at the age of eleven which was managed with the use of olanzapine tablets in 2.5–10 mg doses per day at different times. The patient developed pain and discomfort in her neck within the second week of being put on tablet olanzapine at a dose of 5 mg per day. This was associated with a sustained and abnormal contraction of the neck muscles that would pull her head to the right in an upward direction. These features had persisted for the first three years of her illness with a varying intensity, distress, and dysfunction which would tend to correlate with the dose of olanzapine. Apart from a brief period of around three weeks when she was given tablet trihexyphenidyl 4 mg per day for rigidity in her upper limbs, she was not prescribed any other psychotropic medication. The rigidity showed good response to this medication which was subsequently stopped. The introduction and subsequent withdrawal of this medication did not bring about any change in the sustained abnormal contraction of her neck muscles.\nImprovement and subsequent remission of the mood symptoms of the patient provided the treatment team with an opportunity to stop olanzapine. The discomfort in the neck and the abnormal movement of the neck muscles persisted over the next three months’ period when she was off olanzapine without any significant change, even with a trial of propranolol, trihexyphenidyl, and phenargan injection. Reintroduction of olanzapine (at a dose of 2.5 mg per day) after a gap of three months for the reemergence of some behavioral features led to a slight aggravation of the already existing abnormal movement and posturing of the neck.\nWith improvement in the clinical picture, olanzapine was reduced and stopped. She was put on tablet sodium valproate, 1000 mg per day during this period for the stabilization of her mood when she was also given escitalopram for a period of three months for her depressive features. The patient responded well to this change in medication, but she developed amenorrhea for which no cause was established after a detailed gynecological evaluation. Keeping in mind the possibility of valproate-induced menstrual disturbance, she was shifted to tablet lithium 450 mg per day. The patient was well maintained on this medication for a period of around two years. However, she developed hypothyroidism for which eltroxin was introduced at a dose of 50 micrograms per day.\nDuring this period of two years and seven months, the abnormal contraction of the neck muscles and the abnormal positioning of the head improved slightly, and with the improvement, it would cause less discomfort and interference in her activities. However, these movements failed to disappear completely. Another exacerbation of the mood symptoms in the form of mania warranted a need for the introduction of olanzapine (by a different treatment team) and the patient was reintroduced to 10 mg olanzapine on a daily basis, which led to the current presentation as described earlier.\nAfter the case was seen at our institute, the psychotropic medications were stopped as her mood symptoms had remitted and she was put on tablet tetrabenazine (built up to 75 mg per day in divided doses) with which the patient had started showing some response with an improvement in abnormal movements of the muscles of the neck as well as the back. She is now able to stand with support and can do some daily chores on her own. The pain and discomfort in the back and neck have also reduced.\nDuring the course of the illness, the patient has been investigated for the presence of any neurological illness as the cause of her abnormal movements. Her MRI scan of the brain, serum and urine copper levels, slit lamp microscopy for the KF ring, complete blood count, TLC, DLC, and USG of the abdomen did not reveal any abnormalities. Her thyroid function tests were deranged subsequent to the introduction of tablet lithium carbonate which was restored to normal after the introduction of tablet eltroxin.\nDystonia is a syndrome of sustained muscle contractions that produce twisting and repetitive movements or abnormal postures. The descriptions of the extent and severity of muscle involvement are variable, ranging from intermittent contraction limited to a single body region, to generalized dystonia involving the limbs and axial muscles.\nEver since the introduction of the term, “dystonia” by Oppenhiem in the early part of the twentieth century, it has been an area of focused attention of the neurologists. In 1973, Keegan and Rajput introduced the term, “” to describe drug-induced, sustained muscle spasm causing repetitive movements or abnormal postures. “” was a term introduced by Burke in 1982, the description of which required the presence of chronic dystonia, a history of antipsychotic drug treatment preceding or concurrent with the onset of dystonia, the exclusion of known causes of secondary dystonia by appropriate clinical and laboratory evaluation, and a negative family history of dystonia for definitive diagnosis.\nThe dystonia could be classified based on the region(s) of the body involved. Involvement of isolated regions like the face, neck, and arms would be labeled as focal dystonia, whereas simultaneous involvement of two or more contiguous areas would be called segmental dystonia. When the clinical picture is that of involvement of two or more noncontiguous regions, the label used is, “multifocal” and the involvement of one leg and one other body region makes it the generalized type.\nThe symptoms of tardive dystonia could begin even after a few days or weeks of exposure to the offending agent. Tardive dystonia is prevalent in 0.5–21.6% of the patients who are treated with neuroleptics.\nThe syndrome of tardive dystonia has been reported with most of the typical antipsychotics.[] It has been associated with the atypical antipsychotics, namely risperidone, olanzapine, quetiapine, and aripiprazole. Reports of tardive dystonia developing with the use of atypical antipsychotics have been predominantly in the cases of nonaffective psychosis and in the adult population in the age ranges of the midthirties and forties. Our case is the first case of an affective illness in an adolescent girl developing tardive dystonia on olanzapine. The aggravation of the clinical features with the inadvertent reintroduction of the medication suggests olanzapine is the offending agent. With the growing acceptance of olanzapine as the first-line therapy for the manic phase of bipolar illness and as a mood stabilizer for the maintenance therapy, one needs to be cautious about the emergence of this troublesome adverse effect of this therapy. The patient has shown some response to the introduction of tetrabenazine.'''


# Generate model summary with the new function
generated_summary= summarize_article_with_tweaks(article)

print('Generated Summary:')
print(generated_summary)

In [ ]:
# Define reference summary for comparison (human-written summary)
reference_summary= '''Visit Motivation:
The patient visited due to discomfort in the neck and lower back, restriction of body movements, inability to maintain an erect posture, and requiring assistance in standing and walking.

Admission:
There were no prior admissions recorded.

Patient Information:
The patient is a sixteen-year-old female. Ethnicity, weight, height, family medical history, recent travels, socio-economic context, and occupation are not available.

Patient Medical History:
There are no notable physiological contexts reported. Psychologically, the patient was diagnosed with bipolar affective disorder at the age of eleven, with the first episode being mania. Vaccination history, allergies, exercise frequency, nutrition, sexual history, alcohol consumption, drug usage, and smoking status are not documented.

Surgeries:
There is no history of surgeries.

Symptoms:
The patient reports discomfort in the neck and lower back, restriction of body movements, and inability to maintain an erect posture. Symptoms have been present for the past four months. The head is turned to the right and upwards due to sustained contraction of neck muscles, and there is sideways bending of the back in the lumbar region. Limbs are positioned to support body weight, and standing up from a sitting position affects the symptoms.

Medical Examinations:
No medical examinations are documented.

Diagnosis Tests:
No diagnostic tests are reported.

Treatments:

    The patient has been taking Olanzapine tablets (5 mg per day) daily for the past four months to control exacerbated mental illness. However, treatment has led to pain and discomfort in the neck, sustained and abnormal contraction of neck muscles, and the need for assistance in daily chores. The patient has a history of taking olanzapine doses between 2.5–10 mg per day over the past seven years.

    Additionally, the patient was prescribed Trihexyphenidyl (4 mg per day) for a brief period of around three weeks to address rigidity in the upper limbs. The response to this treatment was good.

Discharge:
No discharge reason, referral, follow-up, or discharge summary has been provided'''

# Load the ROUGE metric using the evaluate library
rouge = evaluate.load('rouge')

# Calculate the ROUGE score
scores= rouge.compute(predictions=[generated_summary], references=[reference_summary])

# Print ROUGE scores
print('\nROUGE scores:')
print(scores)